In [5]:
import urllib
import json

In [38]:
class office_data:
    ask = "https://declarator.org/api/v1/search/sections/?office="
    
    def find_children(self):
        return []

    def __init__(self, office_id):
        self.office_id = office_id
        data_struct = {} 
        data = {'next':self.ask+str(office_id)}
        while data["next"]:
            with urllib.request.urlopen(data["next"]) as url:
                data = json.loads(url.read().decode())
            for i in data["results"]:
                if i["main"]["year"] not in data_struct:
                    data_struct[i["main"]["year"]] = []
                data_struct[i["main"]["year"]].append(i)
            print(data["next"])
        #надо обойти всех child и вызвать одну из функций sort
        child_list = self.find_children()
        for i in child_list:
            child_data = get_data_office(i)
            for j in child_data:
                if j not in data_struct:
                    data_struct[j] = []
                data_struct[j] += child_data[j]

        self.data = data_struct # данные всех деклараций по годам
        self.family = 0 # учитывать ли семью
        
    def true_avg(self, typ):
        years = self.data.keys()
        res = [0]  * len(years)
        for i, year in enumerate(years):
            curr_sum = 0
            curr_amount = 0
            for declaration in self.data[year]:
                for pep in declaration[typ]:
                    inc = pep['size']
                    if inc == inc:
                        curr_amount += 1
                        curr_sum += inc
            if curr_amount != 0:
                res[i] = curr_sum / curr_amount
        for j in range(len(res)):
            if res[j] ==0:
                res[j] = None
        return res
        
    def gender_avg(self, typ):
        years = self.data.keys()
        males = [0] * len(years)
        females = [0] * len(years)
        for i, year in enumerate(years):
            male_sum = 0
            female_sum = 0
            male_amount = 0
            female_amount = 0
            for declaration in self.data[year]:
                gender = declaration['main']['person']['gender']
                for pep in declaration[typ]:
                    inc = pep['size']
                    if inc == inc:
                        if gender == 'M':
                            male_sum += inc
                            male_amount += 1
                        elif gender == 'F':
                            female_sum += inc
                            female_amount += 1
            if male_amount != 0:
                males[i] = male_sum / male_amount
            if female_amount != 0:
                females[i] = female_sum / female_amount
        
        for j in range(len(males)):
            if males[j] ==0:
                males[j] = None
        for j in range(len(females)):
            if females[j] ==0:
                females[j] = None
        return males, females
        
    def party_avg(self, typ):
        parties = set()
        years = self.data.keys()
        for year in years:
            for declaration in self.data[year]:
                party = declaration['main']['party']
                if party != party or party == None:
                    parties.add('Нет Данных')
                else:
                    parties.add(party['name'])
        res = {par : [0]*len(years) for par in parties}
        for i, year in enumerate(years):
            sums = {par : 0 for par in parties}
            amounts = {par : 0 for par in parties}
            for declaration in self.data[year]:
                party = declaration['main']['party']
                if party != party or party == None:
                    party = 'Нет Данных'
                else:
                    party = party['name']
                for pep in declaration[typ]:
                    inc = pep['size']
                    if inc == inc:
                        sums[party] += inc
                        amounts[party] += 1
            for party in parties:
                if amounts[party] != 0:
                    res[party][i] = sums[party] / amounts[party]
        for i in res:
            for j in range(len(res[i])):
                if res[i][j] ==0:
                    res[i][j] = None
        return res
    #self.data - список по годам деклараций людей

In [39]:
office = office_data(12)
# office = office_data(12)

https://declarator.org/api/v1/search/sections/?office=12&page=2
None


In [40]:
office.party_avg("incomes")

{'Нет Данных': [None,
  None,
  None,
  None,
  None,
  None,
  None,
  1698095.99,
  1140930.92,
  816426.2899999999,
  751742.425],
 'Единая Россия': [551047.0,
  605144.0,
  755617.08,
  855212.69,
  1054909.5,
  1365071.5899999999,
  1804028.41,
  None,
  None,
  None,
  None]}